# Introduction

Many imaging studies require T1-weighted images with specific acquisition parameters, usually sagittal acquisition. In PPMI, metadata on MRI acquisition is available in file "Magnetic_Resonance_Imaging__MRI_.csv", however, this file does not contain detailed information about acquistion parameters.

This notebook downloads acquisition parameters of 3D T1-weighted images from the PPMI imaging database, filters sagittal acquisition scans, and converts visit names used in the imaging database to the ones used in other PPMI metadata. 

The resulting file can be used to build imaging cohorts based on PPMI.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the Python code."></form>''')


# Data download

Let's download information about PPMI 3D T1-weighted scans:

In [2]:
import os
import os.path as op
import pandas as pd
import ppmi_downloader

data_dir = op.join('inputs', 'study_files')

if not op.exists(data_dir):
    os.makedirs(data_dir)

mri_file_name = '3D_mri_info.csv'
required_files = [mri_file_name]
missing_files = [x for x in required_files if not op.exists(os.path.join(data_dir, x))]

if len(missing_files) > 0:
    ppmi = ppmi_downloader.PPMIDownloader()
    file_name = ppmi.download_3D_T1_info(destination_dir=data_dir, headless=False)
    assert(op.exists(op.join(data_dir, file_name)))
    os.rename(op.join(data_dir, file_name), op.join(data_dir, mri_file_name))
    ppmi.quit()

print('File downloaded')

PPMI login:  mathdugre@pm.me
PPMI password:  ········


LivingPark-utils|DEBUG|ppmi_downloader.py:106 in __init__()
                       self.tempdir: <TemporaryDirectory '/tmp/tmpuzv0d_at'>
LivingPark-utils|DEBUG|ppmi_downloader.py:117 in __init__()
                       self.config_file: '.ppmi_config'
                       config_file: '.ppmi_config'
LivingPark-utils|DEBUG|ppmi_navigator.py:64 in click_button()
                       'Click button': 'Click button'
                       field: 'ida-cookie-policy-accept'
                       debug_name: 'Cookie Policy'
LivingPark-utils|DEBUG|ppmi_navigator.py:64 in click_button()
                       'Click button': 'Click button'
                       field: 'ida-cookie-policy-accept'
                       debug_name: 'Cookie Policy'
LivingPark-utils|DEBUG|ppmi_navigator.py:33 in wait_for_element_to_be_visible()
                       'Wait for element to be visible': 'Wait for element to be visible'
                       field: 'ida-user-username'
                       BY: '

File downloaded


# Filter sagittal acquisitions

PPMI scans were acquired using the following protocols:

In [3]:
pd.set_option('display.max_rows', 500)
mri_info = pd.read_csv(op.join(data_dir, mri_file_name))
mri_info.groupby('Description').count()

,Subject ID,Sex,Visit,Study Date,Age,Imaging Protocol
Description,,,,,,
3D SAG,2,2,2,2,2,2
3D SAG T1 MPRAGE,4,4,4,4,4,4
3D SAG T1 MPRAGE_ND,3,3,3,3,3,3
3D Sagittal T1,7,7,7,7,7,7
3D T1,11,11,11,11,11,11
3D T1 MPRAGE,14,14,14,14,14,14
3D T1-weighted,53,53,53,53,53,53
AX 3D FSPGR straight brain lab,1,1,1,1,1,1
AX T1,11,11,11,11,11,11


To keep only the sagittal acquisitions, we will remove the following protocols:

In [4]:
# Remove sequences that exactly match the following
removed_sequences=['COR', # coronal acquisitions
                   'Coronal', 
                   'Cal Head 24',  # not sure what this is
                   'Transverse',  # transverse (axial) acquisitions
                   'tra_T1_MPRAGE',
                   'TRA'
                  ]
print(removed_sequences)

['COR', 'Coronal', 'Cal Head 24', 'Transverse', 'tra_T1_MPRAGE', 'TRA']


We will also remove the protocol names containing the following strings:

In [5]:
# Remove sequences containing the following strings
removed_sequences_contain = ['AX', 'axial', 'Phantom']
print(removed_sequences_contain)

['AX', 'axial', 'Phantom']


We obtain the following list of protocols:

In [6]:
mri_info = mri_info[~mri_info['Description'].isin(removed_sequences)]
for s in removed_sequences_contain:
    mri_info = mri_info[~mri_info['Description'].str.contains(s)]
mri_info.groupby('Description').count()

,Subject ID,Sex,Visit,Study Date,Age,Imaging Protocol
Description,,,,,,
3D SAG,2,2,2,2,2,2
3D SAG T1 MPRAGE,4,4,4,4,4,4
3D SAG T1 MPRAGE_ND,3,3,3,3,3,3
3D Sagittal T1,7,7,7,7,7,7
3D T1,11,11,11,11,11,11
3D T1 MPRAGE,14,14,14,14,14,14
3D T1-weighted,53,53,53,53,53,53
Accelerated Sag IR-FSPGR,16,16,16,16,16,16
FSPGR 3D SAG,27,27,27,27,27,27


# Convert visit names

Let's use the following map to convert visit names to the codes used in PPMI metadata:

In [7]:
visit_map = {
    'Screening': 'SC',
    'Baseline': 'BL',
    'Month 12': 'V04',
    'Month 24': 'V06',
    'Month 36': 'V08',
    'Month 48': 'V10',
    'Symptomatic Therapy': 'ST',
    'Unscheduled Visit 01': 'U01',
    'Unscheduled Visit 02': 'U02',
    'Premature Withdrawal': 'PW'
}
print(visit_map)

{'Screening': 'SC', 'Baseline': 'BL', 'Month 12': 'V04', 'Month 24': 'V06', 'Month 36': 'V08', 'Month 48': 'V10', 'Symptomatic Therapy': 'ST', 'Unscheduled Visit 01': 'U01', 'Unscheduled Visit 02': 'U02', 'Premature Withdrawal': 'PW'}


We obtain the following distribution by visit code:

In [8]:
mri_info['Visit code'] = mri_info['Visit'].apply(lambda x: visit_map[x])
mri_info.groupby('Visit code').count()

,Subject ID,Sex,Visit,Study Date,Age,Description,Imaging Protocol
Visit code,,,,,,,
BL,1173,1173,1173,1173,1173,1173,1173
PW,1,1,1,1,1,1,1
SC,4,4,4,4,4,4,4
ST,10,10,10,10,10,10,10
U01,10,10,10,10,10,10,10
U02,2,2,2,2,2,2,2
V04,279,279,279,279,279,279,279
V06,324,324,324,324,324,324,324
V08,2,2,2,2,2,2,2


Finally, let's save our table as csv file:

In [9]:
filename = 'MRI_info.csv'
mri_info.to_csv(op.join(data_dir, filename), index=False)
print(f'Saved in {filename}')

Saved in MRI_info.csv
